### What-If Tool on COMPAS

Copyright 2019 Google LLC.
SPDX-License-Identifier: Apache-2.0

This notebook shows use of the [What-If Tool](https://pair-code.github.io/what-if-tool) on the COMPAS dataset.


For ML fairness background on COMPAS see:
- https://www.propublica.org/article/machine-bias-risk-assessments-in-criminal-sentencing
- https://www.propublica.org/article/how-we-analyzed-the-compas-recidivism-algorithm
- http://www.crj.org/assets/2017/07/9_Machine_bias_rejoinder.pdf

The dataset is from the [COMPAS kaggle page](https://www.kaggle.com/danofer/compass).

This notebook trains a linear classifier on the on the COMPAS dataset to mimic the behavior of the the COMPAS recidivism classifier. We can then analyze our COMPAS proxy model for fairness using the What-If Tool.

The specific binary classification task for this model is to determine if a person belongs in the "Low" risk class according to COMPAS (negative class), or the "Medium" or "High" risk class (positive class).

In [1]:
#@title Install the What-If Tool widget if running in colab {display-mode: "form"}

try:
  import google.colab
  import os
  !pip install --upgrade witwidget
  os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'
  import google.protobuf
  print(f"Protobuf version: {google.protobuf.__version__}")
  print(f"Using PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION={os.environ.get('PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION', 'cpp')}")
except:
  pass

In [2]:
#@title Define helper functions {display-mode: "form"}

import pandas as pd
import numpy as np
import tensorflow as tf
import functools

# Creates a tf feature spec from the dataframe and columns specified.
def create_feature_spec(df, columns=None):
    feature_spec = {}
    if columns == None:
        columns = df.columns.values.tolist()
    for f in columns:
        if df[f].dtype is np.dtype(np.int64):
            feature_spec[f] = tf.io.FixedLenFeature(shape=(), dtype=tf.int64)
        elif df[f].dtype is np.dtype(np.float64):
            feature_spec[f] = tf.io.FixedLenFeature(shape=(), dtype=tf.float32)
        else:
            feature_spec[f] = tf.io.FixedLenFeature(shape=(), dtype=tf.string)
    return feature_spec

# Creates Keras preprocessing layers for features (replaces deprecated feature columns)
# Returns a dictionary mapping feature names to preprocessing layers
def create_preprocessing_layers(columns, feature_spec, df):
    preprocessing_layers = {}
    for col in columns:
        if feature_spec[col].dtype == tf.string:
            # Categorical string feature - use StringLookup with one-hot encoding
            vocab = sorted(list(df[col].dropna().unique()))
            preprocessing_layers[col] = tf.keras.layers.StringLookup(
                vocabulary=vocab, output_mode='one_hot', name=f'{col}_lookup')
        elif feature_spec[col].dtype == tf.int64:
            # Integer feature - convert to float and pass through
            preprocessing_layers[col] = tf.keras.layers.Lambda(
                lambda x: tf.cast(x, tf.float32), name=f'{col}_cast')
        else:
            # Float feature - pass through (or normalize if desired)
            preprocessing_layers[col] = tf.keras.layers.Lambda(
                lambda x: x, name=f'{col}_pass')
    return preprocessing_layers

# An input function for providing input to a model from tf.Examples
# Updated for TensorFlow 2.16+ (no estimator API)
def tfexamples_input_fn(examples, feature_spec, label, mode='eval',
                       num_epochs=None,
                       batch_size=64):
    def ex_generator():
        for i in range(len(examples)):
            yield examples[i].SerializeToString()
    dataset = tf.data.Dataset.from_generator(
      ex_generator, tf.dtypes.string, tf.TensorShape([]))
    if mode == 'train':
        dataset = dataset.shuffle(buffer_size=2 * batch_size + 1)
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(lambda tf_example: parse_tf_example(tf_example, label, feature_spec))
    if num_epochs:
        dataset = dataset.repeat(num_epochs)
    return dataset

# Parses Tf.Example protos into features for the input function.
def parse_tf_example(example_proto, label, feature_spec):
    parsed_features = tf.io.parse_example(serialized=example_proto, features=feature_spec)
    target = parsed_features.pop(label)
    return parsed_features, target

# Converts a dataframe into a list of tf.Example protos.
def df_to_examples(df, columns=None):
    examples = []
    if columns == None:
        columns = df.columns.values.tolist()
    for index, row in df.iterrows():
        example = tf.train.Example()
        for col in columns:
            value = row[col]

            # Skip NaN values
            if pd.isna(value):
                continue

            # Check actual value type, not just column dtype
            if isinstance(value, (int, np.integer)):
                example.features.feature[col].int64_list.value.append(int(value))
            elif isinstance(value, (float, np.floating)):
                example.features.feature[col].float_list.value.append(float(value))
            else:
                # Everything else becomes a string
                example.features.feature[col].bytes_list.value.append(str(value).encode('utf-8'))
        examples.append(example)
    return examples

# # Converts a dataframe into a list of tf.Example protos.
# def df_to_examples(df, columns=None):
#     examples = []
#     if columns == None:
#         columns = df.columns.values.tolist()
#     for index, row in df.iterrows():
#         example = tf.train.Example()
#         for col in columns:
#             if df[col].dtype is np.dtype(np.int64):
#                 example.features.feature[col].int64_list.value.append(int(row[col]))
#             elif df[col].dtype is np.dtype(np.float64):
#                 example.features.feature[col].float_list.value.append(row[col])
#             elif row[col] == row[col]:
#                 example.features.feature[col].bytes_list.value.append(row[col].encode('utf-8'))
#         examples.append(example)
#     return examples

# Converts a dataframe column into a column of 0's and 1's based on the provided test.
# Used to force label columns to be numeric for binary classification using a TF estimator.
def make_label_column_numeric(df, label_column, test):
  df[label_column] = np.where(test(df[label_column]), 1, 0)

C:\Users\Yufan Wang\.conda\envs\compas_wit1\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


In [3]:
print(tf.__version__)
print(hasattr(tf, 'estimator'))

2.13.0
True


In [4]:
#@title Read training dataset from CSV {display-mode: "form"}

import pandas as pd

df = pd.read_csv('https://storage.googleapis.com/what-if-tool-resources/computefest2019/cox-violent-parsed_filt.csv')
df


,id,name,first,last,sex,dob,age,age_cat,race,juv_fel_count,...,vr_charge_desc,type_of_assessment,decile_score.1,score_text,screening_date,v_type_of_assessment,v_decile_score,v_score_text,priors_count.1,event
0,1.0,miguel hernandez,miguel,hernandez,Male,18/04/1947,69,Greater than 45,Other,0,...,NaN,Risk of Recidivism,1,Low,14/08/2013,Risk of Violence,1,Low,0,0
1,2.0,miguel hernandez,miguel,hernandez,Male,18/04/1947,69,Greater than 45,Other,0,...,NaN,Risk of Recidivism,1,Low,14/08/2013,Risk of Violence,1,Low,0,0
2,3.0,michael ryan,michael,ryan,Male,06/02/1985,31,25 - 45,Caucasian,0,...,NaN,Risk of Recidivism,5,Medium,31/12/2014,Risk of Violence,2,Low,0,0
3,4.0,kevon dixon,kevon,dixon,Male,22/01/1982,34,25 - 45,African-American,0,...,Felony Battery (Dom Strang),Risk of Recidivism,3,Low,27/01/2013,Risk of Violence,1,Low,0,1
4,5.0,ed philo,ed,philo,Male,14/05/1991,24,Less than 25,African-American,0,...,NaN,Risk of Recidivism,4,Low,14/04/2013,Risk of Violence,3,Low,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18311,NaN,alexsandra beauchamps,alexsandra,beauchamps,Female,21/12/1984,31,25 - 45,African-American,0,...,NaN,Risk of Recidivism,6,Medium,29/12/2014,Risk of Violence,4,Low,5,0
18312,NaN,winston gregory,winston,gregory,Male,01/10/1958,57,Greater than 45,Other,0,...,NaN,Risk of Recidivism,1,Low,14/01/2014,Risk of Violence,1,Low,0,0
18313,NaN,farrah jean,farrah,jean,Female,17/11/1982,33,25 - 45,African-American,0,...,NaN,Risk of Recidivism,2,Low,09/03/2014,Risk of Violence,2,Low,3,0
18314,NaN,florencia sanmartin,florencia,sanmartin,Female,18/12/1992,23,Less than 25,Hispanic,0,...,NaN,Risk of Recidivism,4,Low,30/06/2014,Risk of Violence,4,Low,2,0


In [5]:
#@title Specify input columns and column to predict {display-mode: "form"}
import numpy as np

# Filter out entries with no indication of recidivism or no compass score
df = df[df['is_recid'] != -1]
df = df[df['decile_score'] != -1]

# Rename recidivism column
df['recidivism_within_2_years'] = df['is_recid']

# Make the COMPASS label column numeric (0 and 1), for use in our model
df['COMPASS_determination'] = np.where(df['score_text'] == 'Low', 0, 1)

# Set column to predict
label_column = 'COMPASS_determination'

# Get list of all columns from the dataset we will use for model input or output.
input_features = ['sex', 'age', 'race', 'priors_count', 'juv_fel_count', 'juv_misd_count', 'juv_other_count']
features_and_labels = input_features + [label_column]

features_for_file = input_features + ['recidivism_within_2_years', 'COMPASS_determination']
features_for_file

['sex',
 'age',
 'race',
 'priors_count',
 'juv_fel_count',
 'juv_misd_count',
 'juv_other_count',
 'recidivism_within_2_years',
 'COMPASS_determination']

In [6]:
#@title Convert dataset to tf.Example protos {display-mode: "form"}

examples = df_to_examples(df, features_for_file)

In [7]:
#@title Create and train the classifier {display-mode: "form"}

num_steps = 2000  #@param {type: "number"}

# Create a feature spec for the classifier
feature_spec = create_feature_spec(df, features_and_labels)

# Create preprocessing layers (replaces deprecated feature columns)
preprocessing_layers = create_preprocessing_layers(input_features, feature_spec, df)

# Build Keras model using modern preprocessing layers (compatible with TensorFlow 2.16+)
# Create input layers
inputs = {}
for col in input_features:
    if feature_spec[col].dtype == tf.string:
        inputs[col] = tf.keras.Input(shape=(), name=col, dtype=tf.string)
    elif feature_spec[col].dtype == tf.int64:
        inputs[col] = tf.keras.Input(shape=(), name=col, dtype=tf.int64)
    else:
        inputs[col] = tf.keras.Input(shape=(), name=col, dtype=tf.float32)

# Apply preprocessing layers and ensure consistent 2D shapes
preprocessed_features = []
for col in input_features:
    preprocessed = preprocessing_layers[col](inputs[col])

    shape_len = len(preprocessed.shape)
    if shape_len == 1:
        preprocessed = tf.keras.layers.Reshape((1,), name=f'{col}_reshape')(preprocessed)
    elif shape_len > 2:
        preprocessed = tf.keras.layers.Flatten(name=f'{col}_flatten')(preprocessed)

    preprocessed_features.append(preprocessed)

feature_layer = tf.keras.layers.Concatenate(axis=-1, name='feature_concat')(preprocessed_features)
output = tf.keras.layers.Dense(1, activation='sigmoid', name='predictions')(feature_layer)
classifier = tf.keras.Model(inputs=inputs, outputs=output)

classifier.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# ========= 🔴 从这里开始是新加的部分，替换掉原来的 tfexamples_input_fn =========

# 把标签列变成 float32
labels = df[label_column].astype('float32').values

# 构造特征字典：保持列名不变，让 Keras 输入和前面定义的 inputs 对得上
features_dict = {}
for col in input_features:
    # 字符串特征保持为字符串数组，数值特征保持为数值
    if df[col].dtype == 'object' or df[col].dtype.name.startswith('string'):
        features_dict[col] = df[col].astype(str).values
    else:
        features_dict[col] = df[col].values

# 用 from_tensor_slices 构造 tf.data.Dataset
batch_size = 64
dataset = tf.data.Dataset.from_tensor_slices((features_dict, labels))
dataset = dataset.shuffle(buffer_size=len(df), reshuffle_each_iteration=True)
train_dataset = dataset.batch(batch_size).repeat()

# 根据 num_steps 计算需要的 epoch 和 steps_per_epoch
dataset_size = len(df)
steps_per_epoch = max(1, dataset_size // batch_size)
epochs = max(1, num_steps // steps_per_epoch)
remaining_steps = num_steps % steps_per_epoch

# 训练模型
if epochs > 0:
    classifier.fit(train_dataset,
                   epochs=epochs,
                   steps_per_epoch=steps_per_epoch,
                   verbose=1)

if remaining_steps > 0:
    classifier.fit(train_dataset,
                   epochs=1,
                   steps_per_epoch=remaining_steps,
                   verbose=1)

print("Model training completed!")


Epoch 1/7
273/273 [==============================] - 1s 1ms/step - loss: 5.7650 - accuracy: 0.4658
Epoch 2/7
273/273 [==============================] - 0s 1ms/step - loss: 1.2809 - accuracy: 0.6077
Epoch 3/7
273/273 [==============================] - 0s 1ms/step - loss: 0.5481 - accuracy: 0.7368
Epoch 4/7
273/273 [==============================] - 0s 1ms/step - loss: 0.5376 - accuracy: 0.7387
Epoch 5/7
273/273 [==============================] - 0s 1ms/step - loss: 0.5274 - accuracy: 0.7441
Epoch 6/7
273/273 [==============================] - 0s 1ms/step - loss: 0.5226 - accuracy: 0.7472
Epoch 7/7
89/89 [==============================] - 0s 1ms/step - loss: 0.5163 - accuracy: 0.7368
Model training completed!


### What-If Tool analysis

We can see the same unfairness that ProPublica found in their analysis by:
1. Going the the "Performance + Fairness" tab
2. Setting "Ground Truth Feature" to "recidivism_within_2_years"
3. In "Slice by" dropdown menu, select "race"
4. Look at the confusion matrices of the "African-American" and "Causasian" slices.
  - They have very similar accuracy (TP+TN)/Total
  - But, the FP rate is MUCH higher for African Americans and the FN rate is MUCH higher for caucasians

In [8]:
#@title Invoke What-If Tool for test data and the trained models {display-mode: "form"}

import os

# Ensure protobuf compatibility
if 'PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION' not in os.environ:
    os.environ['PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION'] = 'python'

num_datapoints = 10000  #@param {type: "number"}
tool_height_in_px = 1000  #@param {type: "number"}

from witwidget.notebook.visualization import WitConfigBuilder
from witwidget.notebook.visualization import WitWidget

# Create a custom prediction function for Keras model
# WIT widget expects a function that takes tf.Example protos and returns predictions
def predict_fn(examples_list):
    """
    Prediction function for WIT widget.
    Takes a list of tf.Example protos and returns predictions.
    """
    # Parse examples into feature dictionaries
    parsed_examples = []
    for example_proto in examples_list:
        # Parse the tf.Example
        parsed = tf.io.parse_example(
            serialized=[example_proto.SerializeToString()],
            features=feature_spec
        )
        # Extract only input features (remove label if present)
        features_dict = {col: parsed[col] for col in input_features if col in parsed}
        parsed_examples.append(features_dict)

    # Convert to batched format for model prediction
    # Model expects dict of {feature_name: tensor} where tensors have batch dimension
    batch_features = {}
    for col in input_features:
        batch_features[col] = tf.stack([ex[col] for ex in parsed_examples])

    # Get predictions from model
    predictions = classifier(batch_features, training=False)

    # Return predictions in format expected by WIT
    # For binary classification, return probabilities for both classes
    # Shape: (batch_size, 2) where [prob_class_0, prob_class_1]
    prob_class_1 = predictions.numpy().flatten()
    prob_class_0 = 1 - prob_class_1
    return tf.stack([prob_class_0, prob_class_1], axis=1).numpy()

# Setup the tool with the test examples and the trained classifier
# Updated for Keras models (TensorFlow 2.16+) using custom prediction function
config_builder = WitConfigBuilder(examples[0:num_datapoints]).set_custom_predict_fn(
    predict_fn)
WitWidget(config_builder, height=tool_height_in_px)

WitWidget(config={'model_type': 'classification', 'label_vocab': [], 'are_sequence_examples': False, 'inferenc…

#### Exploration ideas

- Organize datapoints by "inference score" (can do this through binning or use of scatter plot) to see points ordered by how likely they were determined to re-offend.
  - Select a point near the boundary line (where red points turn to blue points)
  - Find the nearest counterfactual to see a similar person with a different decision. What is different?
  - Look at the partial dependence plots for the selected person. What changes in what features would change the decision on this person?
- In "Performance and Fairness" tab, slice the dataset by different features (such as race or sex)
  - Look at the confusion matrices for each slice - How does performance compare in those slices? What from the training data may have caused the difference in performance between the slices? What root causes could exist?
  - Use the threshold optimization buttons to optimize positive classification thresholds for each slice based on any of the possible fairness constraints - How different do the thresholds have to be to achieve that constraint? How varied are the thresholds depending on the fairness constraint chosen?

- In the "Performance + Fairness" tab, change the cost ratio so that you can optimize the threshold based off of a non-symmetric cost of false positives vs false negatives. Then click the "optimize threshold" button and see the effect on the confusion matrix.
  - Slice the dataset by a feature, such as sex or race. How has the new cost ratio affected the disparity in performance between slices? Click the different threshold optimization buttons to see how the changed cost ratio affects the disparity given different fairness constraints.

- Try adding/removing features from the set of input features that the model uses during training. The model trained by this notebook only uses 7 of the columns from the dataset, as defined in the "Specify input columns and column to predict" cell in this notebook. How does your new set of input features affect the model performance (overall and across slices).

- If you set the ground truth feature to "COMPAS_determination" in the "Performance + Fairness" tab, you will see the confusion matrix and ROC curve of how good our model is at being a proxy for the COMPAS model itself (as opposed to how good our model is at predicting recidivism).
  - How well is our model doing at its task? What types of errors does it have?
  - Try improving the performance of the model. Options include adding more input features, changing the model architecture, and training for more steps.
  - After you've improved our proxy COMPAS model, what (if any) change in unfairness do you see when evaluating against "recidivism_after_2_years"?

